In [1]:
import pandas as pd

### make smaller eqtl and gwas to work with 

In [27]:
def get_clusters(TISSUE, CHROM):
    # Logic to generate a list of sample names
    return open(coloc_output_dir + '{TISSUE}/temp/{TISSUE}.{CHROM}.cluster_list.txt').read().strip().split("\n")


In [3]:
cluster_path = '/home/klawren/oak/pcqtls/output/proteincoding_main/clusters/Adipose_Subcutaneous_clusters_all_chr.csv'

In [9]:
        import pandas as pd
        cluster_df =  pd.read_csv(cluster_path,index_col=0)
        for idx, row in cluster_df.iterrows():
            cluster_df.loc[idx, 'cluster_id']  = '_'.join([*sorted(row['Transcripts'].split(','))])
        output_head = '/home/klawren/oak/pcqtls/output/'
        tissue_id = 'Adipose_Subcutaneous'
        for chrom in cluster_df['Chromosome'].unique():
            cluster_df[cluster_df['Chromosome']==chrom]['cluster_id'].to_csv(f'{output_head}/temp/{tissue_id}.chr{chrom}.cluster_list.txt', index=False, header=False)


In [21]:
eqtl = pd.read_parquet('output/proteincoding_main/control_eqtl/Adipose_Subcutaneous/Adipose_Subcutaneous.v8.cluster_genes.cis_qtl_pairs.chr22.parquet')
cluster_list = pd.read_csv(f'{output_head}/temp/{tissue_id}.chr22.cluster_list.txt', header=None, names=['cluster_id'])

In [22]:
cluster_list

,cluster_id
0,ENSG00000099937.10_ENSG00000099940.11_ENSG0000...
1,ENSG00000099974.7_ENSG00000133433.10_ENSG00000...
2,ENSG00000100342.20_ENSG00000128335.13
3,ENSG00000025708.13_ENSG00000177989.13
4,ENSG00000100429.17_ENSG00000188130.13
5,ENSG00000075240.16_ENSG00000100422.13
6,ENSG00000243811.8_ENSG00000244509.3
7,ENSG00000100221.10_ENSG00000100226.15
8,ENSG00000100151.15_ENSG00000100156.10
9,ENSG00000100139.13_ENSG00000128346.10


In [24]:
eqtl['cluster_id'] = eqtl['phenotype_id'].str.split('_e').str[0]

In [26]:
for idx, row in cluster_list.iterrows():
    snp_list = eqtl[eqtl['cluster_id'] == row['cluster_id']]['variant_id']
    snp_list.to_csv('output/temp/snp_list.csv', index=False, sep='\t')

In [ ]:
for idx, row in eqtl.iterrows():
       eqtl.loc[idx, 'cluster_id']  = row['Transcripts'].split(','))])


In [13]:
sub_eqtl = eqtl[eqtl['phenotype_id'] == eqtl['phenotype_id'].iloc[0]]

In [14]:
sub_eqtl.to_csv('output/temp/test_eqtl.csv', index=None, sep='\t')

In [15]:
gwas = pd.read_csv('/oak/stanford/groups/smontgom/shared/gwas_summary_stats/barbeira_gtex_imputed/imputed_gwas_hg38_1.1/imputed_UKB_50_Standing_height.txt.gz', sep='\t')

In [16]:
gwas.head()

,variant_id,panel_variant_id,chromosome,position,effect_allele,non_effect_allele,current_build,frequency,sample_size,zscore,pvalue,effect_size,standard_error,imputation_status,n_cases
0,rs554008981,chr1_13550_G_A_b38,chr1,13550,A,G,hg38,0.017316,336474,-2.291993,0.021906,NaN,NaN,imputed,NaN
1,rs201055865,chr1_14671_G_C_b38,chr1,14671,C,G,hg38,0.012987,336474,-0.955919,0.339113,NaN,NaN,imputed,NaN
2,rs201327123,chr1_14677_G_A_b38,chr1,14677,A,G,hg38,0.053391,336474,1.495331,0.134828,NaN,NaN,imputed,NaN
3,rs199856693,chr1_14933_G_A_b38,chr1,14933,A,G,hg38,0.011544,336474,-0.461158,0.644685,NaN,NaN,imputed,NaN
4,rs62636368,chr1_16841_G_T_b38,chr1,16841,T,G,hg38,0.045455,336474,0.793825,0.427297,NaN,NaN,imputed,NaN


In [17]:
eqtl['position'] = eqtl['variant_id'].str.split('_').str[1].astype(int)

In [18]:
max_position = sub_eqtl['variant_id'].str.split('_').str[1].astype(int).max()
min_position = sub_eqtl['variant_id'].str.split('_').str[1].astype(int).min()

In [19]:
gwas = gwas[gwas['chromosome'] == 'chr22']
gwas.fillna('NA', inplace=True)
sub_gwas = gwas[(gwas['position'].astype(int) > min_position) & (gwas['position'].astype(int) < max_position)]

In [20]:
snp_list = sub_eqtl[sub_eqtl['variant_id'].isin(sub_gwas['panel_variant_id'])]['variant_id']
snp_list.to_csv('output/temp/snp_list.csv', index=False, sep='\t')

In [21]:
sub_gwas.to_csv('output/temp/test_gwas.csv', index=None, sep='\t')

In [31]:
ld_matrix = pd.read_csv('output/temp/test.ld', sep='\t', header=None, names=snp_list)

In [35]:
ld_matrix.set_index(snp_list, inplace=True)

In [36]:
ld_matrix.isna().sum().sum()

178

In [ ]:
chr22_16956420_C_T_b38

In [47]:
ld_matrix['chr22_16954390_G_A_b38'][ld_matrix['chr22_16954390_G_A_b38'].isna()]

variant_id
chr22_16956420_C_T_b38   NaN
Name: chr22_16954390_G_A_b38, dtype: float64

In [40]:
ld_matrix[ld_matrix.isna().sum()>0]

,chr22_16628911_CTTG_C_b38,chr22_16629375_T_A_b38,chr22_16629512_C_G_b38,chr22_16629863_T_G_b38,chr22_16630596_A_T_b38,chr22_16631452_C_T_b38,chr22_16631637_C_CCAT_b38,chr22_16631746_C_T_b38,chr22_16632220_A_G_b38,chr22_16632936_G_T_b38,...,chr22_18637212_A_G_b38,chr22_18637251_G_T_b38,chr22_18638681_C_CT_b38,chr22_18658980_A_G_b38,chr22_18659236_T_C_b38,chr22_18659263_C_G_b38,chr22_18727775_T_A_b38,chr22_18727816_T_C_b38,chr22_18727918_G_C_b38,chr22_18727946_C_T_b38
variant_id,,,,,,,,,,,,,,,,,,,,,
chr22_16954390_G_A_b38,0.000268,0.001199,0.000510,0.000204,0.001530,0.000207,0.000108,0.000928,0.000003,6.591740e-04,...,0.003536,0.001597,0.000267,0.000027,0.000663,0.000447,0.000008,0.000829,0.001124,0.001018
chr22_16955693_G_A_b38,0.000268,0.001199,0.000510,0.000204,0.001530,0.000207,0.000108,0.000928,0.000003,6.591740e-04,...,0.003536,0.001597,0.000267,0.000027,0.000663,0.000447,0.000008,0.000829,0.001124,0.001018
chr22_16955737_C_T_b38,0.000268,0.001199,0.000510,0.000204,0.001530,0.000207,0.000108,0.000928,0.000003,6.591740e-04,...,0.003536,0.001597,0.000267,0.000027,0.000663,0.000447,0.000008,0.000829,0.001124,0.001018
chr22_16956420_C_T_b38,0.000164,0.000185,0.000485,0.003975,0.001848,0.000495,0.000329,0.000355,0.000102,4.488370e-07,...,0.000100,0.000235,0.000017,0.001307,0.000081,0.000108,0.000302,0.000157,0.003610,0.003775
chr22_16956485_G_C_b38,0.000268,0.001199,0.000510,0.000204,0.001530,0.000207,0.000108,0.000928,0.000003,6.591740e-04,...,0.003536,0.001597,0.000267,0.000027,0.000663,0.000447,0.000008,0.000829,0.001124,0.001018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr22_18132365_C_T_b38,0.002062,0.000443,0.000129,0.000086,0.000549,0.001831,0.000156,0.000268,0.000092,7.427620e-04,...,0.000027,0.000009,0.000918,0.000223,0.002478,0.005141,0.000162,0.000998,0.003670,0.010987
chr22_18135190_T_TG_b38,0.002993,0.000002,0.000132,0.000179,0.000092,0.002614,0.000213,0.000022,0.001065,7.887450e-05,...,0.000962,0.000439,0.000015,0.002364,0.002452,0.002707,0.000908,0.000324,0.000018,0.001248
chr22_18135974_T_C_b38,0.002993,0.000002,0.000132,0.000179,0.000092,0.002614,0.000213,0.000022,0.001065,7.887450e-05,...,0.000962,0.000439,0.000015,0.002364,0.002452,0.002707,0.000908,0.000324,0.000018,0.001248


In [44]:
eqtl[eqtl['variant_id']=='chr22_16956420_C_T_b38']

,phenotype_id,variant_id,start_distance,end_distance,af,ma_samples,ma_count,pval_nominal,slope,slope_se,position
1331,ENSG00000015475.18_ENSG00000099968.17_e_ENSG00...,chr22_16956420_C_T_b38,-672435,-818350,0.374322,373,434,0.391571,-0.002587,0.003017,16956420
8819,ENSG00000015475.18_ENSG00000099968.17_e_ENSG00...,chr22_16956420_C_T_b38,-672435,-818350,0.374322,373,434,0.027248,0.006652,0.003004,16956420
